## Step 0: Defining parameters
Please modify the parameters as follows:
1. `country` that you are working on.

2. `mode` can be one of two values "9to5" or "overnight". Don't forget to use double quotes.

Most of you will only use the "9to5" value. Which will tell the code to only translate 1,000 articles. This amount of articles takes between 3 and 4 hours to run. So this is ideal to use when you are in the office. Some of us (I'm looking at you DAU), will leave the code running the whole night to translate 4,000 articles every night. You are welcome to use the "overnight" mode if you want to finish your batch quickly (although we are not expecting you to finish it... unless you are an active member of the DAU).

3. `counter_day` and `counter_night` are just a counter of how many times you have SUCESSFULLY executed the code using the "9to5" or the "overnight" mode, respectively. If this is the first time you are running the code, both counters should be zero. If you already SUCESSFULLY run the code once using the "9to5" mode, then counter_day should be equal to one. If the code stopped or you had an issue and the code did not finished running, then it DOES NOT count as a SUCESSFUL execution.

4. `folow_up` can take two values True or False. Set it to True if you already finished a translation batch and your session is still running (and your master data is already uploaded). This way you don't need to execute all cells again, only the ones required. If this is the first time you are running this script during your current session, then set this value to False. No double quotes needed.

In [167]:
country       = "Slovenia"
mode          = "overnight" # One of two values: "9to5" OR "overnight"
counter_day   = 2
counter_night = 0
suffix        = "tp" # One of two values: "tp" OR "retrans"
path2SP       = "/Users/ctoruno/OneDrive - World Justice Project/EU Subnational"

## Step 1: Installing and importing required libraries

In [168]:
import os
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize
from deep_translator import GoogleTranslator
nltk.download("punkt")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ctoruno\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Step 2: Defining functions

In [169]:
def trans2english(text, sourcelang):
    """
    This functions gathers a text in a specific language and it returns its equivalent
    in English using the Google translation engine.

    Parameters:
        text:       String. Text to translate.
        sourcelang: String. Code of the source language you want to translate the text from.
    """
    if text:
      try:
          sentences = sent_tokenize(text)
          batch  = GoogleTranslator(source = sourcelang, target = "en").translate_batch(sentences)
          result = " ".join(batch)
          return result
      except Exception as e:
          out = f"Translation through API failed. Reason: {e}"
          return out
    else:
      return "No information available. No translation performed"

## Step 3: Reading the data

In [170]:
master_data = pd.read_parquet(f"{path2SP}/EU-S Data/Automated Qualitative Checks/Data/data-extraction-1/data4translation/{country}_{suffix}.parquet.gzip")

## Step 4: Subsetting the data

In [171]:
batch = counter_day + counter_night + 1
if mode == "9to5":
    batch_size   = 500
    mode_counter = counter_day
if mode == "overnight":
    batch_size   = 2000
    mode_counter = counter_night

starting_row = (counter_day*500)+(counter_night*2000)
final_row    = starting_row+batch_size
batch_subset = master_data.copy().iloc[starting_row:final_row]

In [172]:
print(starting_row)
print(final_row)
print(master_data.shape)

1000
3000
(10211, 10)


In [173]:
batch_subset.head(3)

,id,link,domain_url,published_date,title,description,content,language,is_opinion,country
1000,348fe17b77ebf248623cf6731f5b675c,https://www.delo.si/novice/svet/zdi-se-da-so-n...,delo.si,2023-10-14 11:29:14,"Zdi se, da so novozelandci izbrali desnosredin...",Novi Zelandiji se obeta sprememba na čelu vlad...,Po približno 80 odstotkih preštetih glasov se ...,sl,False,Slovenia
1001,521ce80e1723be95cfb91631881642a9,https://www.delo.si/novice/svet/na-referendumu...,delo.si,2023-10-14 08:00:00,Na referendumu o pravicah aboriginov,Jih bodo po 122 letih vključili v ustavo in ji...,V Avstraliji danes volivci na zgodovinskem ref...,sl,False,Slovenia
1002,8ab616fec00733437f268e8b0e33a05a,https://www.delo.si/kultura/oder/sad-ljubezni-...,delo.si,2023-10-14 07:30:00,Sad ljubezni do odrskih desk in druženja,Na jubilejni Novačanov festival ljubiteljskih ...,Celjsko gledališče Zarja bo prihodnji teden od...,sl,False,Slovenia


## Step 5: Translating headline, description, and content

In [174]:
batch_subset[["title_trans", "description_trans", "content_trans"]] = batch_subset.apply(
    lambda row: row[["title", "description", "content"]].apply(lambda x: trans2english(text = x, sourcelang = row["language"])),
    axis = 1
)

## Step 6: Saving batch data

In [ ]:
batch_subset.to_parquet(f"/users/ctoruno/Downloads/{country}_batch_{batch}_{mode}_{mode_counter}.parquet.gzip", compression = "gzip")

In [ ]:
batch_subset["content_trans"].str.contains("API").value_counts()

content_trans
True     355
False    145
Name: count, dtype: int64